# Commodity Forecasting - Пример использования

Демонстрация работы с библиотекой прогнозирования цен на сырьевые товары.

In [25]:
import sys
sys.path.insert(0, '..')

from forecasting import MLForecaster, KalmanForecaster

## 1. ML Forecaster

In [26]:
ml = MLForecaster(config='../configs/sugar.yaml')
ml.fit(data_path="../data/sugar_factors.xlsx")

Загрузка данных из ../data/sugar_factors.xlsx...
Загружено строк: 222
Базовые признаки (6): ['Stock-to-Use', 'Brent', 'DXY', 'Baltic Dry Index', 'US CPI', 'USDBRL']
Создание лагов: [3, 6, 9, 12, 18, 24]
Создание MA: [3, 6, 9, 12]
Всего признаков после генерации: 66
Режим: ЭКСПЕРТНЫЙ ВЫБОР
Выбрано признаков: 4
  1. Stock-to-Use
  2. Brent
  3. DXY
  4. Baltic Dry Index

Разделение данных:
  Train: 166 (75.1%)
  Val: 33 (14.9%)
  Test: 22 (10.0%)

ОБУЧЕНИЕ МОДЕЛЕЙ

──────────────────────────────────────────────────
Модель: CATBOOST
──────────────────────────────────────────────────
  MAE: 57.1631
  RMSE: 69.06
  MAPE: 10.45%

──────────────────────────────────────────────────
Модель: XGB
──────────────────────────────────────────────────
  MAE: 47.1185
  RMSE: 60.91
  MAPE: 8.85%

──────────────────────────────────────────────────
Модель: LGB
──────────────────────────────────────────────────
  MAE: 55.8697
  RMSE: 63.38
  MAPE: 10.50%

──────────────────────────────────────────────────


In [27]:
ml.save_results()
ml.plot_results()


Сохранение результатов...
  Результаты: results/sugar
  Модели: saved_models/sugar
Графики сохранены в results/sugar


In [28]:
ml.summary()


ИТОГИ ML FORECASTING

Model                MAE        RMSE       MAPE       R²        
------------------------------------------------------------
★xgb                 47.119     60.905     8.85       -0.3016   
 ridge               52.504     61.382     9.33       -0.3221   
 lgb                 55.870     63.377     10.50      -0.4094   
 catboost            57.163     69.058     10.45      -0.6734   
 random_forest       58.985     68.422     10.81      -0.6427   
------------------------------------------------------------

🏆 Лучшая модель: xgb
   MAE: 47.119


In [29]:
print(f"Лучшая модель: {ml.best_model}")
print(f"Метрики: {ml.metrics[ml.best_model]}")
print(f"\nВсе метрики:")
for model, m in ml.metrics.items():
    print(f"  {model}: MAE={m['mae']:.3f}, MAPE={m['mape']:.2f}%")

Лучшая модель: xgb
Метрики: {'mae': 47.11854568522992, 'rmse': np.float64(60.9054598667037), 'mape': np.float64(8.846884121721201), 'r2': -0.3016119331653164}

Все метрики:
  catboost: MAE=57.163, MAPE=10.45%
  xgb: MAE=47.119, MAPE=8.85%
  lgb: MAE=55.870, MAPE=10.50%
  ridge: MAE=52.504, MAPE=9.33%
  random_forest: MAE=58.985, MAPE=10.81%


## 2. Kalman Forecaster

In [30]:
kf = KalmanForecaster(config='../configs/sugar.yaml')
kf.fit(data_path="../data/sugar_factors.xlsx")

Загрузка данных из ../data/sugar_factors.xlsx...
Загружено строк: 221
Признаки (6): ['Stock-to-Use', 'Brent', 'DXY', 'Baltic Dry Index', 'US CPI', 'USDBRL']

Разделение данных:
  Train: 199 (90.0%)
  Test: 22 (10.0%)

ОБУЧЕНИЕ TIME-VARYING KALMAN FILTER

Метрики на Train:
  MAE: 0.022
  MAPE: 0.00%

Тест Дики-Фуллера: p-value = 0.0000

Прогнозирование методами:
  - last_beta
  - random_walk
  - linear_trend
  - ar_beta
  - damped_trend
  - bayesian_shrinkage
  - adaptive_ensemble

🏆 Лучший метод: bayesian_shrinkage (MAE = 51.8708)


In [31]:
kf.save_results()
kf.plot_results()


Сохранение результатов...
  Результаты: results/sugar
  Модели: saved_models/sugar
Графики сохранены в results/sugar


In [32]:
kf.summary()


ИТОГИ KALMAN FORECASTING

Method                    MAE        RMSE       MAPE       R²        
-----------------------------------------------------------------
★bayesian_shrinkage       51.871     58.044     9.68       -0.1822   
 ar_beta                  62.395     67.101     11.66      -0.5799   
 adaptive_ensemble        90.828     95.203     17.02      -2.1803   
 last_beta                101.773    106.401    19.13      -2.9725   
 random_walk              101.773    106.401    19.13      -2.9725   
 damped_trend             136.430    142.937    25.79      -6.1690   
 linear_trend             304.109    323.567    57.88      -35.7366  
-----------------------------------------------------------------

🏆 Лучший метод: bayesian_shrinkage
   MAE: 51.871


In [33]:
print(f"Лучший метод: {kf.best_method}")
print(f"Метрики: {kf.metrics[kf.best_method]}")
print(f"\nВсе метрики:")
for method, m in kf.metrics.items():
    print(f"  {method}: MAE={m['mae']:.3f}, MAPE={m['mape']:.2f}%")

Лучший метод: bayesian_shrinkage
Метрики: {'mae': 51.87082869578796, 'rmse': np.float64(58.04372391621103), 'mape': np.float64(9.683131708972677), 'r2': -0.18216910065839453}

Все метрики:
  last_beta: MAE=101.773, MAPE=19.13%
  random_walk: MAE=101.773, MAPE=19.13%
  linear_trend: MAE=304.109, MAPE=57.88%
  ar_beta: MAE=62.395, MAPE=11.66%
  damped_trend: MAE=136.430, MAPE=25.79%
  bayesian_shrinkage: MAE=51.871, MAPE=9.68%
  adaptive_ensemble: MAE=90.828, MAPE=17.02%


## 3. Сравнение моделей

In [34]:
print("="*60)
print("СРАВНЕНИЕ ML vs KALMAN")
print("="*60)

ml_best_mae = ml.metrics[ml.best_model]['mae']
kf_best_mae = kf.metrics[kf.best_method]['mae']

print(f"\nML ({ml.best_model}): MAE = {ml_best_mae:.3f}")
print(f"Kalman ({kf.best_method}): MAE = {kf_best_mae:.3f}")

if ml_best_mae < kf_best_mae:
    print(f"\nML лучше на {(kf_best_mae - ml_best_mae):.3f}")
else:
    print(f"\nKalman лучше на {(ml_best_mae - kf_best_mae):.3f}")

ml_best_mape = ml.metrics[ml.best_model]['mape']
kf_best_mape = kf.metrics[kf.best_method]['mape']

print(f"\nML ({ml.best_model}): MAPE = {ml_best_mape:.3f}")
print(f"Kalman ({kf.best_method}): MAPE = {kf_best_mape:.3f}")

if ml_best_mape < kf_best_mape:
    print(f"\nML лучше на {(kf_best_mape - ml_best_mape):.3f}")
else:
    print(f"\nKalman лучше на {(ml_best_mape - kf_best_mape):.3f}")

СРАВНЕНИЕ ML vs KALMAN

ML (xgb): MAE = 47.119
Kalman (bayesian_shrinkage): MAE = 51.871

ML лучше на 4.752

ML (xgb): MAPE = 8.847
Kalman (bayesian_shrinkage): MAPE = 9.683

ML лучше на 0.836


## 4. Использование без YAML конфига

In [35]:
ml_custom = MLForecaster(
    input_file='../data/sugar_factors.xlsx',
    target_column='Sugar Price',
    date_column='Date',
    models_to_train=['catboost', 'xgb'],
    n_optuna_trials=50,
    test_size_ratio=0.1,
    results_dir='../results/custom',
    models_dir='../saved_models/custom'
)
# ml_custom.fit()